In [ ]:
import os
import pandas as pd
from datetime import datetime as dt
import time
import plotly.express as px
import mysql.connector
from mysql.connector import Error
import numpy as np
import config

# Connecting to Wordpress Database

In [ ]:
host = config.host
database = config.database
user = config.user
password = config.password

conn = mysql.connector.connect(host = host, database = database, user = user, password = password)

# Reading wp_jet_appointment Wordpress table

#### SQL query set to appointments

In [ ]:
appointments = "SELECT user_id, status, service, provider, user_email, date, slot, slot_end \
                FROM wp_jet_appointments \
                ORDER BY slot asc"

#### Reading sql query and saving to csv

In [ ]:
ap_data = pd.read_sql_query(appointments, conn)
ap_data.to_csv("assets/data/wp_jet_appointments.csv", index=False)